# **Leer data transformadores**

In [ ]:
import pandas as pd

# Lee el archivo CSV
df_transformadores = pd.read_csv('df_rellenado_con_nans.csv')

# Muestra las primeras filas del DataFrame
df_transformadores.head()

In [ ]:
df_transformadores['estacion_2'].unique()

In [ ]:
columnas = [
    'Fecha y hora',
    "SE_San_Rafael.Trf_San_Rafael_T1 Potencia activa media (kW)",
    "SE_San_Rafael.Trf_San_Rafael_T1 Potencia reactiva media (kVAr)",
    "SE_San_Rafael.Trf_San_Rafael_T3 Potencia activa media (kW)",
    "SE_San_Rafael.Trf_San_Rafael_T3 Potencia reactiva media (kVAr)",
    "SE_San_Felipe.Trf_San_Felipe_T1 Potencia activa media (kW)",
    "SE_San_Felipe.Trf_San_Felipe_T1 Potencia reactiva media (kVAr)",
    "SE_San_Felipe.Trf_San_Felipe_T2 Potencia activa media (kW)",
    "SE_San_Felipe.Trf_San_Felipe_T2 Potencia reactiva media (kVAr)",
    "SE_Concon.Trf_Concon_T2_2 Potencia activa media (kW)",
    "SE_Concon.Trf_Concon_T2_2 Potencia reactiva media (kVAr)",
    'estacion',
    'estacion_2'
]

df_transformadores = df_transformadores[columnas]

df_transformadores['Fecha y hora'] = pd.to_datetime(df_transformadores['Fecha y hora'])


In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
import os

transformador_columnas_p = [
 "SE_San_Rafael.Trf_San_Rafael_T1 Potencia activa media (kW)",
 "SE_San_Rafael.Trf_San_Rafael_T3 Potencia activa media (kW)",
 "SE_San_Felipe.Trf_San_Felipe_T1 Potencia activa media (kW)",
 "SE_San_Felipe.Trf_San_Felipe_T2 Potencia activa media (kW)", 
 "SE_Concon.Trf_Concon_T2_2 Potencia activa media (kW)"
]
    

transformador_columnas_q = [
 "SE_San_Rafael.Trf_San_Rafael_T1 Potencia reactiva media (kVAr)", 
 "SE_San_Rafael.Trf_San_Rafael_T3 Potencia reactiva media (kVAr)", 
 "SE_San_Felipe.Trf_San_Felipe_T1 Potencia reactiva media (kVAr)",
 "SE_San_Felipe.Trf_San_Felipe_T2 Potencia reactiva media (kVAr)",
"SE_Concon.Trf_Concon_T2_2 Potencia reactiva media (kVAr)"
]

In [ ]:
# Unir ambas listas de columnas
columnas_a_convertir = transformador_columnas_p + transformador_columnas_q

# Aplicar la conversión: dividir por 1000
df_transformadores[columnas_a_convertir] = df_transformadores[columnas_a_convertir] / 1000

In [ ]:
df_transformadores.head()

In [ ]:
df_transformadores['Fecha y hora'] = pd.to_datetime(df_transformadores['Fecha y hora'])

# Mostrar la fecha mínima y máxima
fecha_min = df_transformadores['Fecha y hora'].min()
fecha_max = df_transformadores['Fecha y hora'].max()

print("Fecha mínima:", fecha_min)
print("Fecha máxima:", fecha_max)


In [ ]:
# Definir las fechas de inicio y fin
fecha_inicio = pd.to_datetime('2022-09-01 00:00:00')
fecha_fin = pd.to_datetime('2025-04-08 00:00:00')

# Filtrar el DataFrame para que solo contenga registros entre fecha_inicio y fecha_fin
df_transformadores = df_transformadores[(df_transformadores['Fecha y hora'] >= fecha_inicio) & (df_transformadores['Fecha y hora'] <= fecha_fin)]

In [ ]:
df_transformadores.info()

# **Instalación de librerias**

In [ ]:
!pip install prophet

In [ ]:
from prophet import Prophet
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# **Modelos Invierten Q**

In [ ]:
def calcular_mape(y_true, y_pred, min_denominador=1.0):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true >= min_denominador
    if np.any(mask):
        return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    else:
        return np.nan

## pruebas feriados (GAP 3 weeks)

In [35]:
def ejecutar_modelo_prophet_parametros_estacion_manual(df, columna, fecha_inicio_test, mostrar_grafico=False, ruta_guardado=None):
    # Preparar datos
    df_prophet = df[['Fecha y hora', columna, 'estacion']].dropna().rename(columns={
        'Fecha y hora': 'ds', columna: 'y'
    })
    df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])

    # Convertir 'estacion' en dummies
    df_prophet = pd.get_dummies(df_prophet, columns=['estacion'], prefix='est')
    columnas_estacion = [col for col in df_prophet.columns if col.startswith('est_')]

    # Fechas clave
    fecha_inicio_test = pd.to_datetime(fecha_inicio_test)
    fecha_fin_test = fecha_inicio_test + pd.Timedelta(days=1)
    
    fecha_fin_train_eval = fecha_inicio_test - pd.Timedelta(weeks=3)
    fecha_inicio_train = fecha_fin_train_eval - pd.Timedelta(weeks=52)
    fecha_inicio_train_eval = fecha_fin_train_eval - pd.Timedelta(days=1)
    
    # Filtrar conjuntos
    df_test = df_prophet[(df_prophet['ds'] >= fecha_inicio_test) & (df_prophet['ds'] < fecha_fin_test)].copy()
    df_train_eval = df_prophet[(df_prophet['ds'] >= fecha_inicio_train_eval) & (df_prophet['ds'] < fecha_fin_train_eval)].copy()
    df_train = df_prophet[df_prophet['ds'] <= fecha_fin_train_eval].copy()

    # Crear y configurar el modelo
    m = Prophet(
        daily_seasonality=False,
        weekly_seasonality=False,
        yearly_seasonality=False,
        n_changepoints=50
    )
    m.add_seasonality(name='daily_hour', period=1, fourier_order=20)
    m.add_seasonality(name='weekly_custom', period=7, fourier_order=30)
    m.add_seasonality(name='yearly_custom', period=365.25, fourier_order=10)

    for col in columnas_estacion:
        m.add_regressor(col)

    # Entrenamiento
    m.fit(df_train)

    # Preparar fechas futuras
    future = pd.concat([df_train_eval[['ds']], df_test[['ds']]]).drop_duplicates().sort_values('ds')
    future = future.merge(df_prophet[['ds'] + columnas_estacion], on='ds', how='left')

    # Predicción
    forecast = m.predict(future)
    forecast_train_eval = forecast[forecast['ds'].isin(df_train_eval['ds'])]
    forecast_test = forecast[forecast['ds'].isin(df_test['ds'])]

    df_train_eval = df_train_eval.set_index('ds').join(forecast_train_eval.set_index('ds')[['yhat']], how='inner')
    df_test_eval = df_test.set_index('ds').join(forecast_test.set_index('ds')[['yhat']], how='inner')

    # Métricas
    mae_train = mean_absolute_error(df_train_eval['y'], df_train_eval['yhat'])
    rmse_train = np.sqrt(mean_squared_error(df_train_eval['y'], df_train_eval['yhat']))
    mape_train = calcular_mape(df_train_eval['y'], df_train_eval['yhat'])

    mae_test = mean_absolute_error(df_test_eval['y'], df_test_eval['yhat'])
    rmse_test = np.sqrt(mean_squared_error(df_test_eval['y'], df_test_eval['yhat']))
    mape_test = calcular_mape(df_test_eval['y'], df_test_eval['yhat'])

    # Gráfico opcional
    if mostrar_grafico or ruta_guardado:
        plt.figure(figsize=(12, 4))
        plt.plot(df_test_eval.index, df_test_eval['y'], label='Real')
        plt.plot(df_test_eval.index, df_test_eval['yhat'], label='Predicción')
        plt.title(f'{columna} [{fecha_inicio_test.date()}]')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()

        if ruta_guardado:
            plt.savefig(ruta_guardado)
        if mostrar_grafico:
            plt.show()
        else:
            plt.close()

    # Armar resultados de métricas
    resultados_metricas = {
        'transformador': columna,
        'fecha_inicio_test': fecha_inicio_test.date(),
        'MAE_train': mae_train, 'RMSE_train': rmse_train, 'MAPE_train': mape_train,
        'MAE_test': mae_test, 'RMSE_test': rmse_test, 'MAPE_test': mape_test
    }

    # Preparar df_test_eval para exportar
    df_test_eval = df_test_eval.reset_index()
    df_test_eval['transformador'] = columna
    df_test_eval = df_test_eval.rename(columns={'y': 'real', 'yhat': 'predicho'})

    return resultados_metricas, df_test_eval


In [37]:
import os
import pandas as pd
import holidays
from pathlib import Path

# -------------------------------------------------------------------
# 1)  Carpeta donde guardar los PNG
# -------------------------------------------------------------------
carpeta_graficos = Path("graficos_resultados_festivos_CL")
carpeta_graficos.mkdir(exist_ok=True)

# -------------------------------------------------------------------
# 2) solo 2024
# -------------------------------------------------------------------

fecha_inicio = pd.Timestamp("2024-04-01")
fecha_fin    = pd.Timestamp("2025-03-30")

years = list(range(fecha_inicio.year, fecha_fin.year + 1))

print(f"Años considerados para feriados: {years}")
print(f"Rango de fechas: {fecha_inicio.date()} hasta {fecha_fin.date()}")


# -------------------------------------------------------------------
# 3)  Construir feriados solo para esos años  ←  (igual que antes)
# -------------------------------------------------------------------
feriados_cl = holidays.CountryHoliday("CL", years=years)

fechas_en_rango = [
    pd.Timestamp(d)
    for d in feriados_cl
    if fecha_inicio <= pd.Timestamp(d) <= fecha_fin
]

print("Feriados en rango filtrados:")
for f in fechas_en_rango:
    print(f)

fechas_test_por_festivo = {}
for fecha in fechas_en_rango:
    anio = fecha.year
    fechas_test_por_festivo.setdefault(anio, []).append(fecha)


# -------------------------------------------------------------------
# 4)  Bucle principal – sin cambios …
# -------------------------------------------------------------------
resultados = []
predicciones_todas = []

for transformador in transformador_columnas_q:
    for anio, fechas in fechas_test_por_festivo.items():
        for fecha in fechas:
            # --- verificación rápida de datos válidos (opcional) ---
            sub_df = df_transformadores[df_transformadores['Fecha y hora'].dt.date == fecha.date()]
            if sub_df[transformador].dropna().shape[0] < 2:
                print(f"Saltando {transformador}, festivo {fecha.date()}: <2 datos válidos")
                continue
            # --------------------------------------------------------

            try:
                fecha_str = fecha.strftime("%Y-%m-%d")
                nombre_archivo = (
                    transformador.replace(" ", "_")
                                 .replace(".", "_")
                                 .replace("(", "")
                                 .replace(")", "")
                                 .replace("/", "_")
                )
                ruta_grafico = carpeta_graficos / f"{nombre_archivo}_{fecha_str}.png"

                resultado, predicciones_test = ejecutar_modelo_prophet_parametros_estacion_manual(
                    df_transformadores,
                    columna=transformador,
                    fecha_inicio_test=fecha,
                    mostrar_grafico=False,
                    ruta_guardado=ruta_grafico
                )

                resultado.update({
                    "transformador": transformador,
                    "fecha_test":    fecha,
                    "festivo":       feriados_cl.get(fecha),
                    "anio":          anio
                })
                resultados.append(resultado)

                predicciones_test["fecha_test"] = fecha
                predicciones_test["festivo"]    = feriados_cl.get(fecha)
                predicciones_test["anio"]       = anio
                predicciones_todas.append(predicciones_test)

            except Exception as e:
                print(f"Error en {transformador}, festivo {fecha}: {e}")

# -------------------------------------------------------------------
# 5)  Consolidar resultados
# -------------------------------------------------------------------
df_resultados   = pd.DataFrame(resultados)
df_predicciones = pd.concat(predicciones_todas, ignore_index=True)

print(df_resultados.head())
print(df_predicciones.head())

Años considerados para feriados: [2024, 2025]
Rango de fechas: 2024-04-01 hasta 2025-03-30
Feriados en rango filtrados:
2024-05-01 00:00:00
2024-05-21 00:00:00
2024-06-20 00:00:00
2024-06-29 00:00:00
2024-07-16 00:00:00
2024-08-15 00:00:00
2024-09-18 00:00:00
2024-09-19 00:00:00
2024-09-20 00:00:00
2024-10-12 00:00:00
2024-10-31 00:00:00
2024-11-01 00:00:00
2024-12-08 00:00:00
2024-12-25 00:00:00
2025-01-01 00:00:00


14:39:16 - cmdstanpy - INFO - Chain [1] start processing
14:40:18 - cmdstanpy - INFO - Chain [1] done processing
14:40:20 - cmdstanpy - INFO - Chain [1] start processing
14:41:39 - cmdstanpy - INFO - Chain [1] done processing
14:41:42 - cmdstanpy - INFO - Chain [1] start processing
14:42:53 - cmdstanpy - INFO - Chain [1] done processing
14:42:56 - cmdstanpy - INFO - Chain [1] start processing
14:45:04 - cmdstanpy - INFO - Chain [1] done processing
14:45:08 - cmdstanpy - INFO - Chain [1] start processing
14:46:58 - cmdstanpy - INFO - Chain [1] done processing
14:47:01 - cmdstanpy - INFO - Chain [1] start processing
14:49:53 - cmdstanpy - INFO - Chain [1] done processing
14:49:56 - cmdstanpy - INFO - Chain [1] start processing
14:52:39 - cmdstanpy - INFO - Chain [1] done processing
14:52:42 - cmdstanpy - INFO - Chain [1] start processing
14:56:05 - cmdstanpy - INFO - Chain [1] done processing
14:56:08 - cmdstanpy - INFO - Chain [1] start processing
14:58:34 - cmdstanpy - INFO - Chain [1]

                                       transformador fecha_inicio_test  \
0  SE_San_Rafael.Trf_San_Rafael_T1 Potencia react...        2024-05-01   
1  SE_San_Rafael.Trf_San_Rafael_T1 Potencia react...        2024-05-21   
2  SE_San_Rafael.Trf_San_Rafael_T1 Potencia react...        2024-06-20   
3  SE_San_Rafael.Trf_San_Rafael_T1 Potencia react...        2024-06-29   
4  SE_San_Rafael.Trf_San_Rafael_T1 Potencia react...        2024-07-16   

   MAE_train  RMSE_train  MAPE_train  MAE_test  RMSE_test   MAPE_test  \
0   0.806526    0.915320   27.504228  0.948224   1.073254   49.888879   
1   0.505912    0.675702   26.352083  2.633050   2.873819  239.670442   
2   0.685615    0.758064   14.734994  3.399753   3.624362  193.286541   
3   1.748671    1.933062   93.070653  3.095984   3.261473  193.751296   
4   0.900856    1.001684   48.467979  0.606459   0.698293   37.370444   

  fecha_test                                festivo  anio  
0 2024-05-01               Día Nacional del Trabajo  202

In [38]:
with pd.ExcelWriter("resultados_prophet_Q_no_invierte_festivos.xlsx", engine="openpyxl") as writer:
    df_resultados.to_excel(writer, sheet_name="Metricas", index=False)
    df_predicciones.to_excel(writer, sheet_name="Predicciones", index=False)